In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hemul import heaan
import hemul.HEAAN as he
from muxcnn.resnet_HEAAN import ResNetHEAAN
from muxcnn.utils import load_params, load_img
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv
from muxcnn.utils import get_channel_last, get_conv_params
from muxcnn.hecnn_par import MultParPack
from muxcnn.hecnn_par import forward_convbn_par

## Torch model

In [3]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

img_tensor = load_img("./cute.jpg", hi=32, wi=32)

# MuxedCNN

In [4]:
imgl = get_channel_last(img_tensor[0].detach().numpy())
ki = 1 # initial ki
hi, wi, ch = imgl.shape

In [5]:
muxed_model = ResNet_MuxConv(model, alpha=12)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


# FHE Context

In [6]:
rot_l = [2**i for i in range(15)]

# prepare for all rotations
rot_l = rot_l + [2**15-1, 
         2**15-33, 2**15-32, 2**15-31,
         2**15-17, 2**15-16, 2**15-15, 
         2**15-9,2**15-8, 2**15-7] + [3,5,7,9,15,17, 31, 33]

hec = heaan.HEAANContext(15, 30, 600, boot=False, load_sk=True, rot_l=rot_l)

Initializing the scheme as the data owner
Loading a secret key from:  ./SecretKey.txt
loading secret key done.
HEAAN CKKS setup is ready 


In [11]:
fhemodel = ResNetHEAAN(model, hec)

In [12]:
ctxt = fhemodel.pack_img_ctxt(img_tensor)

decrypted = hec.decrypt(ctxt)

# forward early

In [13]:
_, ins0, outs0 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})

In [14]:
# FHE
cts = fhemodel.forward_convbn_par_fhe(model.conv1, 
                          model.bn1, ctxt, ins0)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
i1,i2, lrots 0 0 33
Rotation by -33
= Rotation by 32735
Rotation by 32735
ct
 1
i1,i2, lrots 0 1 32
Rotation by -32
= Rotation by 32736
Rotation by 32736
ct
 2
i1,i2, lrots 0 2 31
Rotation by -31
= Rotation by 32737
Rotation by 32737
ct
 3
ct
 1
i1,i2, lrots 1 0 1
Rotation by -1
= Rotation by 32767
Rotation by 32767
ct
 1
i1,i2, lrots 1 1 0
ct
 2
i1,i2, lrots 1 2 -1
Rotation by 1
Rotation by 1
ct
 3
ct
 2
i1,i2, lrots 2 0 -31
Rotation by 31
Rotation by 31
ct
 1
i1,i2, lrots 2 1 -32
Rotation by 32
Rotation by 32
ct
 2
i1,i2, lrots 2 2 -33
Rotation by 33
Rotation by 33
ct
 3
ct
 3
aaaaa
bbbbbb
30 570
cccccc
xxxxxxxxxx
ct[i1][i2]
 30
w [0. 0. 0. ... 0. 0. 0.]
dddddd
eeeeee
fffff
xxxxxxxxxx
ct[i1][i2]
 30
w [0. 0. 0. ... 0. 0. 0.]
dddddd
eeeeee
fffff
xxxxxxxxxx
ct[i1][i2]
 30
w [0. 0. 0. ... 0. 0. 0.]
dddddd
eeeeee
fffff
xxxxxxxxxx
ct[i1][i2]
 30
w [0.        

In [ ]:
cts

In [13]:
print(ctxt._arr[2000:2050] == out1[2000:2050])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


In [5]:
# early conv and bn
_, ins0, outs0 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins0)

In [6]:
# Muxed
U, ins1, outs1 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins1)

out1, un1 = forward_convbn_par(model.conv1, 
                              model.bn1, ct_a, ins1)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
2.7468 s


## Activation

In [24]:
from hemul.comparator_fhe import ApprRelu_FHE
from hemul.comparator import ApprRelu

In [31]:
appr_relu = ApprRelu(xmin=-10, xmax=10, min_depth=True)
out1 = appr_relu(out1)
print(out1)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02
[ 1.23083757e+00  4.86967574e-01  2.75344729e-01 ... -5.07712600e-10
  7.09781745e-10  1.15660812e-10]


In [36]:
appr = ApprRelu_FHE(ev, xmin=-10, xmax=10, min_depth=True)
activated = appr(ctxt)
print(activated._arr)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02
[ 1.23083757e+00  4.86967574e-01  2.75344729e-01 ... -5.07710431e-10
  7.09780806e-10  1.15659451e-10]


In [43]:
np.all(np.isclose(out1,activated._arr))

True

In [ ]:
def forward_early(self, img_tensor):
    model = self.torch_model
    imgl = get_channel_last(img_tensor[0].detach().numpy())
    ki = 1 # initial ki
    hi, wi, ch = imgl.shape

    # early conv and bn
    _, ins0, outs0 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
    ct_a = MultParPack(imgl, ins0)
    ctxt, un1 = forward_convbn_par(model.conv1, 
                                   model.bn1, ct_a, ins0)
    ctxt = self.activation(ctxt)
    return ctxt, outs0 

In [21]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8150,  -3.1036,  -2.9026,  -9.4616,   4.3280,  -6.9752,
         -3.5619,  -8.8842,  -7.5630], dtype=torch.float64)

In [10]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8150,  -3.1036,  -2.9026,  -9.4616,   4.3280,  -6.9752,
         -3.5619,  -8.8842,  -7.5630], dtype=torch.float64)